In [1]:
import nest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import elephant
import quantities as pq
import neo
import sys
from scipy.stats import ks_2samp
plt.rcParams['figure.figsize'] = (14, 10)
import scipy.stats as stats
import itertools

In [4]:
cv_exc = pd.read_csv('brunel_round_1_2_5min_CV_exc.csv',index_col=0)
cv_inh = pd.read_csv('brunel_round_1_2_5min_CV_inh.csv',index_col=0)

In [6]:
cv_inh

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0.179416,0.177757,0.178995,0.175276,0.180078,0.176074,0.173874,0.177085,0.177964,0.177059,...,0.177650,0.178591,0.177359,0.181268,0.177224,0.179034,0.174315,0.177997,0.175847,0.174274
1,0.180397,0.179086,0.173525,0.172497,0.176090,0.178199,0.177965,0.179303,0.179345,0.177552,...,0.176983,0.175036,0.176405,0.176074,0.176942,0.176026,0.176540,0.178531,0.179380,0.174940
2,0.177551,0.177025,0.177486,0.180877,0.180715,0.177431,0.176239,0.175652,0.175264,0.175828,...,0.178826,0.174688,0.175806,0.177923,0.179636,0.175744,0.181104,0.176454,0.176676,0.180383
3,0.176399,0.178725,0.178620,0.177508,0.178235,0.179318,0.176277,0.176380,0.175440,0.174051,...,0.173156,0.177675,0.178789,0.178393,0.176669,0.177212,0.181347,0.176579,0.176957,0.174356
4,0.177889,0.179997,0.176453,0.178149,0.175206,0.175116,0.175586,0.176660,0.178554,0.178023,...,0.177103,0.172649,0.177100,0.180959,0.173619,0.175727,0.172437,0.175890,0.178002,0.174468
5,0.175688,0.180071,0.177053,0.178648,0.173850,0.180134,0.176871,0.179182,0.177402,0.181409,...,0.177634,0.177872,0.178372,0.179365,0.174475,0.178791,0.179658,0.179584,0.178603,0.175941
6,0.181125,0.179622,0.175537,0.178704,0.174172,0.175725,0.176317,0.174498,0.174324,0.178422,...,0.171862,0.177442,0.179450,0.175375,0.178159,0.176284,0.179293,0.181535,0.179273,0.177012
7,0.174794,0.172818,0.175179,0.176745,0.177389,0.179534,0.176002,0.180160,0.177861,0.179857,...,0.176145,0.176691,0.177168,0.175066,0.174303,0.178905,0.180935,0.175567,0.177392,0.179197
8,0.177087,0.179973,0.178030,0.180552,0.180203,0.175511,0.174531,0.182037,0.177863,0.179309,...,0.178537,0.177229,0.176119,0.176100,0.175660,0.175096,0.176184,0.175452,0.175906,0.179417
9,0.176090,0.180986,0.178036,0.173911,0.179036,0.178799,0.175341,0.176578,0.180566,0.176267,...,0.179555,0.175274,0.176300,0.180645,0.180527,0.176328,0.178174,0.175214,0.176807,0.178624


In [10]:
def ks_test(value_list):
    """
    Perform KS-test on all possible pairs of simulations from same model. (10 simulations). 
    Return statistic and p-values.
    """
    statistics = []
    p_values = []
    for i in range(0,10):
        for j in range(0,i):
            statistic = ks_2samp(value_list.iloc[i], value_list.iloc[j], alternative='two-sided')[0]
            p_value = ks_2samp(value_list.iloc[i], value_list.iloc[j], alternative='two-sided')[1]
            p_values.append(p_value)
            statistics.append(statistic)
    return statistics, p_values

In [11]:
def num_fail_pass(statistics, p_values):
    """
    Get number of tests which failed and which passed.
    """
    failed = 0
    passed = 0
    for i in range(0,len(statistics)):
        if statistics[i] > p_values[i]:
            # test fails
            failed += 1
        else:
            passed += 1
    s = str(passed) + ' tests passed, ' + str(failed) + ' tests failed.'
    return s

In [12]:
statistics, p_values = ks_test(cv_exc)
s = num_fail_pass(statistics, p_values)
print(s)

21 tests passed, 24 tests failed.


Worse results. Try to calculate simulation to simulation correlation.

In [27]:
def cc(value_list):
    """
    Get correlation of simulation to simulation cv distributions.
    Return one correlation value for each possible pair of simulations (45).
    """
    ccs = []
    for i in range(0,10):
        for j in range(0,i):
            cc = np.corrcoef(value_list.iloc[i], value_list.iloc[j])
            ccs.append(cc[0,1])           
    return ccs

In [28]:
cc_exc = cc(cv_exc)
cc_inh = cc(cv_inh)

In [30]:
print('Mean CC: ',np.mean(cc_exc),' ,standard deviation: ', np.std(cc_exc))
print('Mean CC: ',np.mean(cc_inh),' ,standard deviation: ', np.std(cc_inh))

Mean CC:  0.0011729589003437668  ,standard deviation:  0.009598178843379442
Mean CC:  0.0017779115116738683  ,standard deviation:  0.019168443211799068
